create the results table for the accessibility paper

In [18]:
import geopandas
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [27]:
perimeters = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    index='id'
)

regions = perimeters[
    perimeters.index.isin([
        'zurich', 'ebc_zrh_v01_ex_zurich', 'zurich_downtown'
    ])
]

regions

,geometry
id,
zurich,"POLYGON ((2685056.896 1244517.810, 2684280.851..."
ebc_zrh_v01_ex_zurich,"MULTIPOLYGON (((2671482.755 1238555.241, 26714..."
zurich_downtown,"POLYGON ((2682089.309 1250107.592, 2683611.456..."


In [28]:
accessibility = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'accessibility_diff.gpkg')
)

accessibility = gpd.sjoin(
    accessibility,
    regions,
    how='left',
    predicate='within'
).rename(columns={'index_right': 'region'})

accessibility

,index,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility_before,accessibility_cycling_before,accessibility_foot_before,...,accessibility_private_cars_diff,accessibility_private_cars_diff_perc,accessibility_transit_diff,accessibility_transit_diff_perc,accessibility_cycling_diff,accessibility_cycling_diff_perc,accessibility_foot_diff,accessibility_foot_diff_perc,geometry,region
0,0,38,1,1,301,3,68672447.0,449.042529,574.198628,227.824450,...,-162.928731,-0.271954,0.0,0.0,72.349107,0.126000,-0.438864,-0.001926,POINT (2686663.001 1244743.001),ebc_zrh_v01_ex_zurich
1,1,48,2,2,-2,-2,68672447.0,449.042045,574.198628,227.824450,...,-162.928731,-0.271954,0.0,0.0,72.349107,0.126000,-0.438864,-0.001926,POINT (2686744.001 1244666.001),ebc_zrh_v01_ex_zurich
2,2,56,1,2,201,2,68672447.0,449.041551,574.198628,227.824450,...,-162.928731,-0.271954,0.0,0.0,72.349107,0.126000,-0.438864,-0.001926,POINT (2686703.001 1244729.001),ebc_zrh_v01_ex_zurich
3,3,13,1,1,-2,-2,69702513.0,331.593569,404.511378,138.711689,...,27.871188,0.055322,0.0,0.0,11.134704,0.027526,0.079790,0.000575,POINT (2697032.001 1251290.001),ebc_zrh_v01_ex_zurich
4,4,20,2,1,-2,-2,69702513.0,331.593459,404.511378,138.711689,...,27.871188,0.055322,0.0,0.0,11.134704,0.027526,0.079790,0.000575,POINT (2696996.001 1251309.001),ebc_zrh_v01_ex_zurich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40692,6603,0,2,1,-2,-2,68222474.0,506.383773,687.447518,296.087981,...,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878,POINT (2682164.001 1247428.001),zurich
40692,6603,0,2,1,-2,-2,68222474.0,506.383773,687.447518,296.087981,...,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878,POINT (2682164.001 1247428.001),zurich_downtown
40693,6604,49,1,2,202,2,68222474.0,506.383387,687.447518,296.087981,...,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878,POINT (2682249.001 1247438.001),zurich
40693,6604,49,1,2,202,2,68222474.0,506.383387,687.447518,296.087981,...,-186.584144,-0.269320,0.0,0.0,49.234949,0.071620,0.852137,0.002878,POINT (2682249.001 1247438.001),zurich_downtown


Add group labels

In [31]:
accessibility['x'] = 'all'
accessibility['y'] = 'all'

accessibility['age_group'] = accessibility.apply(
    lambda row: 'age<=25' if row['age'] <= 25 else 'age>=65' if row['age'] >= 65 else 'age:other',
    axis=1
)

accessibility['nationality'] = accessibility.apply(
    lambda row: 'nationality:swiss' if row['residencepermit'] <= -2 else 'nationality:other',
    axis=1
)

accessibility['maritalstatus_type'] = accessibility.apply(
    lambda row: 'maritalstatus:married' if row['maritalstatus'] == 2 else 'maritalstatus:other',
    axis=1
)

accessibility['sex_type'] = accessibility.apply(
    lambda row: 'sex:male' if row['sex'] == 1 else 'sex:female' if row['sex'] == 2 else 'sex:other',
    axis=1
)

stats_values = ['median']

fields = {
    'x': ['count'],
    f'accessibility_{MODE_PRIVATE_CARS}_before': stats_values,
    f'accessibility_{MODE_TRANSIT}_before': stats_values,
    f'accessibility_{MODE_CYCLING}_before': stats_values,
    f'accessibility_{MODE_FOOT}_before': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}_after': stats_values,
    f'accessibility_{MODE_TRANSIT}_after': stats_values,
    f'accessibility_{MODE_CYCLING}_after': stats_values,
    f'accessibility_{MODE_FOOT}_after': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}_diff_perc': stats_values,
    f'accessibility_{MODE_TRANSIT}_diff_perc': stats_values,
    f'accessibility_{MODE_CYCLING}_diff_perc': stats_values,
    f'accessibility_{MODE_FOOT}_diff_perc': stats_values,
    f'accessibility_before': stats_values,
    f'accessibility_after': stats_values,
    f'accessibility_diff_perc': stats_values,
}

accessibility_stats = {
    'region_and_age': accessibility.groupby(['region', 'age_group']).agg(fields),
    'region_and_nationality': accessibility.groupby(['region', 'nationality']).agg(fields),
    'region_and_sex': accessibility.groupby(['region', 'sex_type']).agg(fields),
    'all': accessibility.groupby(['x', 'y']).agg(fields)
}

res = pd.concat(accessibility_stats.values())
res = res.iloc[::-1]

#for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
#    res[f'accessibility_{mode}_diff'] = (
#        res[f'accessibility_{mode}_diff'] / res[f'accessibility_{mode}_before']
#    )
    
#res[f'accessibility_diff'] = (
#        res[f'accessibility_diff'] / res[f'accessibility_before']
#    )

res = res.sort_index().transpose()

res.to_excel(
    os.path.join(outputs_path, 'accessibility_results_transposed.xlsx'),
)

res

all  \
                                                      all   
x                                    count   43783.000000   
accessibility_private_cars_before    median    590.596097   
accessibility_transit_before         median    347.837638   
accessibility_cycling_before         median    501.788854   
accessibility_foot_before            median    173.270281   
accessibility_private_cars_after     median    590.088622   
accessibility_transit_after          median    347.837638   
accessibility_cycling_after          median    525.837637   
accessibility_foot_after             median    173.506949   
accessibility_private_cars_diff_perc median      0.035151   
accessibility_transit_diff_perc      median      0.000000   
accessibility_cycling_diff_perc      median      0.045294   
accessibility_foot_diff_perc         median      0.000669   
accessibility_before                 median    399.231550   
accessibility_after                  median    416.829660   
accessibility_diff_perc              median      0.027664   

                                            ebc_zrh_v01_ex_zurich  \
                                                        age:other   
x                                    count           16430.000000   
accessibility_private_cars_before    median            566.111920   
accessibility_transit_before         median            319.985000   
accessibility_cycling_before         median            450.371466   
accessibility_foot_before            median            152.704214   
accessibility_private_cars_after     median            598.177175   
accessibility_transit_after          median            319.985000   
accessibility_cycling_after          median            464.825813   
accessibility_foot_after             median            152.762026   
accessibility_private_cars_diff_perc median              0.055297   
accessibility_transit_diff_perc      median              0.000000   
accessibility_cycling_diff_perc      median              0.033042   
accessibility_foot_diff_perc         median              0.000502   
accessibility_before                 median            380.064429   
accessibility_after                  median            398.865477   
accessibility_diff_perc              median              0.041052   

                                                                       \
                                                 age<=25      age>=65   
x                                    count   8120.000000  5209.000000   
accessibility_private_cars_before    median   566.884712   564.384818   
accessibility_transit_before         median   318.909210   318.741625   
accessibility_cycling_before         median   447.740313   447.249750   
accessibility_foot_before            median   152.131863   151.675832   
accessibility_private_cars_after     median   598.586817   594.232373   
accessibility_transit_after          median   318.909210   318.741625   
accessibility_cycling_after          median   462.269449   462.802355   
accessibility_foot_after             median   152.202747   151.737236   
accessibility_private_cars_diff_perc median     0.055965     0.053331   
accessibility_transit_diff_perc      median     0.000000     0.000000   
accessibility_cycling_diff_perc      median     0.032743     0.032562   
accessibility_foot_diff_perc         median     0.000508     0.000478   
accessibility_before                 median   379.333807   378.140319   
accessibility_after                  median   398.638513   396.119860   
accessibility_diff_perc              median     0.041634     0.038970   

                                                               \
                                            nationality:other   
x                                    count        7883.000000   
accessibility_private_cars_before    median        575.068469   
accessibility_transit_before         median        329.577513   
accessibility_cycling_before         median        465.821900   
accessi